# Summary
[MONAI](https://docs.monai.io/en/stable/index.html) provides many useful tools to develop AI deep learning models for various tasks in medical imaging. Additionally, we recommend using [Pytorch Lightning](https://lightning.ai/pytorch-lightning) for streamlining your experiment.

This notebook demonstrates a complete workflow for classification of 2D images of the MedNIST dataset. Features demonstrated include:
* Downloading and preparing data from MedNIST for training and validation.
* Using MONAI Transforms to homogenous data and perform data augmentations.
* Setup Pytorch Lightning for streamlined training and testing.
* Using predefined convolutional neural networks from MONAI to classify the 2D images.

# Note

Many more tutorials using MONAI in other tasks, such as segmentation and registration, as well as extension to 3D can be found [here](https://github.com/Project-MONAI/tutorials/tree/main)

# Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[pillow, tqdm]"
!python -c "import sklearn" || pip install -q "scikit-learn"
!python -c "import lightning" || pip install -q "lightning"
!python -c "import matplotlib" || pip install -q "matplotlib"

# Setup imports

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from monai.apps import download_and_extract
from monai.metrics import ROCAUCMetric
from monai.utils import first
from monai.data import DataLoader, Dataset
from monai.networks.nets import DenseNet121
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    LoadImaged,
    ScaleIntensityd,
    AsDiscreted
)
from monai.visualize import matshow3d

# Download data
The MedNIST dataset was compiled by combining various subsets sourced from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset.

Dr. Bradley J. Erickson M.D., Ph.D. from the Department of Radiology at Mayo Clinic generously provides access to this dataset under the Creative Commons CC BY-SA 4.0 license.

When utilizing the MedNIST dataset, it's important to provide proper attribution to the source.

In [3]:
# Set the root_dir for downloading MedNIST
root_dir = os.getcwd()

In [4]:
resource = "https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz"
md5 = "0bc7306e7427e00ad1c5526a6677552d"

compressed_file = Path(root_dir, "MedNIST.tar.gz")
data_dir = Path(root_dir, "MedNIST")
if data_dir.exists:
    download_and_extract(resource, compressed_file, root_dir, md5)

2023-08-24 11:12:16,276 - INFO - Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
2023-08-24 11:12:16,277 - INFO - File exists: /home/dspaanderman/model_development_2_models/Notebooks/MedNIST.tar.gz, skipped downloading.
2023-08-24 11:12:16,277 - INFO - Non-empty folder exists in /home/dspaanderman/model_development_2_models/Notebooks/MedNIST, skipped extracting.


# Create readable datastructure for MONAI

There are 6 folders in the dataset, AbdomenCT, BreastMRI, ChestCT, CXR, Hand, HeadCT, these also correspond to the label. Now we first are going to create a list of dictionaries to match the dataset structure typically used in MONAI. https://docs.monai.io/en/stable/data.html#dataset

In [5]:
# First we get all the classes available in the MedNIST, e.g. AbdomenCT
class_names = sorted(x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, x)))
class_len = len(class_names)
class_mapping = {class_names[i]: i for i in range(class_len)}

data = [
    [{"image": image, "label": class_idx} for image in (data_dir / class_names[class_idx]).glob("*.jpeg")] 
    for class_idx in range(class_len)
    ]

data = [item for sublist in data for item in sublist]

In [ ]:
# Lets print the first 5 items in the data structure to see what it looks like
data[0:5]

In [6]:
# And we have the following labels
class_mapping

{'AbdomenCT': 0,
 'BreastMRI': 1,
 'CXR': 2,
 'ChestCT': 3,
 'Hand': 4,
 'HeadCT': 5}

While we are working on the data, lets also create a training and test split using sklearn. Here we use a 80% train and 20% test split. Also note that we set a random state. This is important if we want to repeat our experiments.

In [7]:
train, val = train_test_split(data, test_size=0.20, random_state=10)

# Transforms

Before we can classify our images we first have to do a couple of things:
- Loading the jpeg into memory
- Normalize the data between the samples, here we do this by scaling the intensity values in the image
- Change the labels to one hot encoding 

In [8]:
transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        ScaleIntensityd(keys=["image"]),
        AsDiscreted(keys=["label"], to_onehot=class_len)
    ]
)

/home/dspaanderman/model_development_2_models/.venv/lib/python3.9/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


# Dataset / Dataloader

In [9]:
# create a training data loader
train_ds = Dataset(data=train, transform=transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4, pin_memory=torch.cuda.is_available())

# create a validation data loader
val_ds = Dataset(data=val, transform=transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=4, pin_memory=torch.cuda.is_available())

Using MONAI we can check the first sample in the dataloader, and visualize it. This is useful when adding new transforms, as you can see what happens if you add a certain transform.

In [10]:
data = first(train_loader)

In [11]:
matshow3d(
    volume=data["image"][:,:,:,:,np.newaxis], #This is done, as we are working with 2D images and the function is originally designed for 3D images 
    fig=None,
    title="input image",
    figsize=(100, 100),
    frame_dim=-1,
    show=True,
    cmap="gray",
)

(<Figure size 10000x10000 with 1 Axes>,
 array([[       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan, 0.09090909, 0.09090909, ..., 0.10588235, 0.11372549,
                nan],
        [       nan, 0.09090909, 0.09090909, ..., 0.10980392, 0.05490196,
                nan],
        ...,
        [       nan, 0.09090909, 0.09090909, ..., 0.09411765, 0.11764706,
                nan],
        [       nan, 0.09090909, 0.09090909, ..., 0.08627451, 0.09803922,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]], dtype=float32))

# Pytorch lightning for model training and testing

Pytorch lightning does most of the heavy lifting regarding model training and testing. We only have to adjust several things:
- Which model to use, e.g. DenseNet121
- Which loss function, e.g. CrossEntropyLoss
- Set a optimizer, e.g. Adam
- Create the training and test step
- Calculate a metric to keep track of learning

In [12]:
class MedNISTModel(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        # Here we define the model used for classification, e.g. DenseNet121
        self.model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=num_classes)

        # Here we can define the loss function we want to use
        self.loss_function = torch.nn.CrossEntropyLoss()

        # Define activation function for prediction
        self.activation_function = Activations(softmax=True)

        # Set a Metric for validation set
        self.metric = ROCAUCMetric()
        self.best_val_AUC = 0
        self.best_val_epoch = 0
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        # This is the training step, done at each epoch, we load the image and label
        x, y = batch["image"], batch["label"]
        # We run the image forward through the model (DenseNet121)
        y_hat = self.forward(x)
        # We calculate the loss based on the output of the model and the true labels
        loss = self.loss_function(y_hat, y)
        # We report the loss function
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # We do the first step similar to training
        x, y = batch["image"], batch["label"]
        y_hat = self.forward(x)
        loss = self.loss_function(y_hat, y)

        # Additionally we can calculate the metric (AUC on the ROC) for each step of the validation
        y_hat = self.activation_function(y_hat)
        self.metric(y_pred=y_hat, y=y)
        self.log('val_loss', loss)
        return loss

    def on_validation_epoch_end(self):
        # Now when a complete validation epoch has done we can report the AUC on the whole set
        val_AUC = self.metric.aggregate()
        self.metric.reset()
        if val_AUC > self.best_val_AUC:
            self.best_val_AUC = val_AUC
            self.best_val_epoch = self.current_epoch
        print(
            f"current epoch: {self.current_epoch} "
            f"current AUC: {val_AUC:.4f}"
            f"\nbest val AUC: {self.best_val_AUC:.4f} "
            f"at epoch: {self.best_val_epoch}"
        )
        self.log("current epoch", self.current_epoch, on_epoch=True)
        self.log("current AUC", val_AUC, on_epoch=True)
        self.log("best AUC", self.best_val_AUC, on_epoch=True)
        self.log("at epoch", self.best_val_epoch, on_epoch=True)
        return val_AUC

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

In [13]:
max_epochs = 4 # number of epochs (whole iterations over the dataset) you would like to train for

model = MedNISTModel(num_classes=class_len) # We defined class_len earlier
trainer = pl.Trainer(max_epochs=max_epochs, accelerator="auto" if torch.cuda.is_available() else "CPU")
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dspaanderman/model_development_2_models/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | DenseNet121      | 7.0 M 
1 | loss_function | CrossEntropyLoss | 0     
----------------

Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.95it/s]current epoch: 0 current AUC: nan
best val AUC: 0.0000 at epoch: 0


/home/dspaanderman/model_development_2_models/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/dspaanderman/model_development_2_models/.venv/lib/python3.9/site-packages/monai/metrics/rocauc.py:83: UserWarning: y values can not be all 0.0, skip AUC computation and return `Nan`.
  warnings.warn(f"y values can not be all {y_unique.item()}, skip AUC computation and return `Nan`.")
/home/dspaanderman/model_development_2_models/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('current epoch', ...)` in your `on_validation_epoch_end` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
/home/dspaanderman/model_development_2_models/.venv

Epoch 0: 100%|██████████| 23582/23582 [36:25<00:00, 10.79it/s, v_num=26]current epoch: 0 current AUC: 0.9253
best val AUC: 0.9253 at epoch: 0
Epoch 0: 100%|██████████| 23582/23582 [39:52<00:00,  9.86it/s, v_num=26]

/home/dspaanderman/model_development_2_models/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 1: 100%|██████████| 23582/23582 [38:25<00:00, 10.23it/s, v_num=26]current epoch: 1 current AUC: 0.9257
best val AUC: 0.9257 at epoch: 1
Epoch 2: 100%|██████████| 23582/23582 [30:30<00:00, 12.88it/s, v_num=26]current epoch: 2 current AUC: 0.9257
best val AUC: 0.9257 at epoch: 1
Epoch 3: 100%|██████████| 23582/23582 [28:10<00:00, 13.95it/s, v_num=26]current epoch: 3 current AUC: 0.9249
best val AUC: 0.9257 at epoch: 1
Epoch 3: 100%|██████████| 23582/23582 [31:11<00:00, 12.60it/s, v_num=26]

`Trainer.fit` stopped: `max_epochs=4` reached.


Epoch 3: 100%|██████████| 23582/23582 [31:11<00:00, 12.60it/s, v_num=26]


# Exercise

Lets try to improve the results generated by the classification model. There are many methods of boosting model performance, lets look at a few.

## Augmentations

First, MONAI comes with many ways to perform data augmentation. These augmentations can be done by adding random transformations. *Note* that you should never to augmentations for the test/validation part of the dataset. Augmentation transforms can be found in the transforms documentation and always start with Rand (for Random), e.g. RandRotated (randomly rotating the image). Find more here: https://docs.monai.io/en/stable/transforms.html#dictionary-transforms

Most random transforms have the argument prob (probability), this is an important argument as it determines the probability of the transform to be performed (e.g. prob=1 means that the transform is always performed).

In [ ]:
train_transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        ScaleIntensityd(keys=["image"]),
        AsDiscreted(keys=["label"], to_onehot=class_len),
        ## EXERCISE - add transform, e.g. RandRotated(keys=["image"], prob=0.5)
    ]
)

# Val and Test transforms should not have any augmentation!
val_transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        ScaleIntensityd(keys=["image"]),
        AsDiscreted(keys=["label"], to_onehot=class_len)
    ]
)

In [ ]:
# create a training data loader
train_ds = Dataset(data=train, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4, pin_memory=torch.cuda.is_available())

# create a validation data loader
val_ds = Dataset(data=val, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=4, pin_memory=torch.cuda.is_available())

It is good while testing augmentation to set probability to 1 and plotting the first image as done before. This will make sure that you are able to see the impact of the augmentation step.

In [ ]:
data = first(train_loader)

matshow3d(
    volume=data["image"][:,:,:,:,np.newaxis], #This is done, as we are working with 2D images and the function is originally designed for 3D images 
    fig=None,
    title="input image",
    figsize=(100, 100),
    frame_dim=-1,
    show=True,
    cmap="gray",
)

## Change classification model

We have now used the DenseNet121, however there are many more models available in MONAI. Tryout another model described here, https://docs.monai.io/en/stable/networks.html#nets

In [ ]:
# Hint, when changing something in a class function, you don't have to copy the whole class function or go back and change it
# MedNISTModel.model = ...

## Hyperparameters

Many different decisions we made in our classification method can be changed or have parameters we could tune. Let's try some:
- Tuning the batch size (e.g. number of samples during each training step).

In [ ]:
# Hint, see batch_size in train_loader

- Training for more epochs (Note don't set it to high, e.g. > 10, otherwise it might take a very long time to run)

In [ ]:
# Hint, see max epochs

- Using another optimizer, try stochastic gradient descent instead of Adam. Note that the learning rate is also a parameter which you can tune, and is ofter larger than the one used in Adam (e.g. 0.01, instead of 0.0001)

In [ ]:
# Hint, similar to the exercise with changing the model, you can substitute configure_optimizers(self)